# 🐸 <font color="#40be46">  JFrog Swampup 2024 MLOPs LAB - The Frog-Factor Authenticator </font> 🐸 

Welcome to the Lab! 


At JFrog, we're always exploring new features and capabilities. Today, we're diving into the authentication market with a brilliant idea: the "Frog-Factor" authenticator. This unique tool will authenticate you by recognizing your face alongside the JFrog frog in the same photo!

Your mission, if you choose to accept it, is to help us build the Frog-Factor authenticator.

First up, we'll need an object-detection model to get us started. Fortunately, we don't have to start from scratch! There are existing models we can use. But will they work out of the box? And remember, we must develop this authenticator securely and reliably.

As you work through the notebook, follow the cells in order:

✨ - This icon means there's a task for you to complete before moving to the next cell.

👀 - This icon provides information about what the next cell is doing.

Let's get started! We're here to help you every step of the way.

✨ <font color="#f8c76b"> TASK </font> ✨

In the cell below, please replace /<USERID> with the userid you got for the labs. For example, if you are user3, it should look like this:

userid = "user3"

In [ ]:
userid = <USERID>

# 🐸  <font color="#40be46">  Lab1: Caching HuggingFace models in Artifactory </font> 🐸 

## Configure the HuggingFace client to work through Artifactory

We don't have to start from scratch!
Luckily HuggingFace contains some great object detection models we can try out. 
Since we want to store the models in Artifactory, we'll need to configure the environment as follows.

✨ <font color="#f8c76b"> TASK </font> ✨

1. Log in to the Artifactory training instance here: https://mlops17234464260.jfrog.io/
2. From the projects dropdown list, select your project ("mlops-userx")
3. Navigate to *Administration --> Repositories*.
4. Click 'Create a Repository' and select *Remote*. Then, select *HuggingFaceML* .
5. On the next page, you only need to provide the *Repository Key* (the repo name)  **The repository will be prefixed with your project name ("mlops-userx"). Please add the repository key "hf-remote"**  then click `Create Remote Repository`
6. A dialog will open, suggesting to set up the HuggingFaceML client or Do It Later - click `Set Up HuggingFaceML client`.
7. Enter your Artifactory password then click `Generate Token & Create Instructions`.
8. Copy the *token* and paste it into the cell below, replacing the \<IDENTITY_TOKEN> placeholder.

👀 The next cell sets the environment variables such that the huggingface client which we'll use later does not fetch the model from the hugging_face hub, but rather from Artifactory.

In [ ]:
# Replace the <IDENTITY-TOKEN> placeholder with the token you generated in the JFrog Platform SetMeUp.
%env HF_TOKEN=<IDENTITY-TOKEN>

%env HF_ENDPOINT=https://mlops17234464260.jfrog.io/artifactory/api/huggingfaceml/mlops-$userid-hf-remote

%env HF_HUB_ETAG_TIMEOUT=86400

## Download the required Python packages through Artifactory

To use and test our model, we'll need some Python packages. Since we want to make sure we're using trusted and allowed packages, we'll get the packages from Artifactory.
We'll configure the Python installations to go to fetch the packages from Artifactory.

We've already configured a Pypi repository for you to use. Run the next cell to download the required dependencies. 

✨ <font color="#f8c76b"> TASK </font> ✨

1. In Artifactory, go to our pypi repository: https://mlops17234464260.jfrog.io/ui/repos/tree/General/mlops-training-remote-pypi 
2. Click `Set Me Up` in the top bar, enter your Artifactory password then click `Generate Token & Create Instructions`. Then, click the `Install` tab.
3. Copy the URL from the value of index-url (starting with "https://...")
![](https://drive.google.com/uc?id=1Y6AiOkyb4P3EUg0YrLpHU-HSg7-2g4cu)
4. Replace \<ARTIFACTORY_PIP_REPOSITORY_URL> with the URL you copied.


🚩 **NOTE:** The following cell may take up to 3 minutes to complete. 
 
🚩  **The cell will show a warning at the end, requesting to restart the session. It is not required, please select `cancel` and continue as usual** 

In [ ]:
# Replace <ARTIFACTORY_PIP_REPOSITORY_URL> with the URL pointing to your pip repository found in the the JFrog Platform Set-Me-Up.
!pip3 install qwak-sdk huggingface_hub ultralytics -i <ARTIFACTORY_PIP_REPOSITORY_URL>

## Python imports

In [ ]:
from huggingface_hub import snapshot_download, HfApi
from huggingface_hub.utils import HfHubHTTPError

import json
import random
from ultralytics import YOLO

import cv2

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

from google.colab.patches import cv2_imshow

import logging,shutil

## Download the pre-trained model

👀 We'll be using the [Yolov8](https://docs.ultralytics.com/) object detection pre-trained model. It's initially configured to only detect human faces.

✨ <font color="#f8c76b"> TASK </font> ✨

Replace \<YOUR_NAME> with your name. Don't forget to put it in quotes, e.g. "Tom". 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Load the model and processor
model_name = "shirabendor/YOLOV8-oiv7"
weights = "yolov8m-oiv7.pt"
config_file = "./model/main/config.json"
name = "<YOUR_NAME>"

try:
    snapshot_download(repo_id=model_name, allow_patterns=[weights, "mlops.zip"], local_dir=".")
except HfHubHTTPError as e:
    print(e)
# unpack the other course materials and remove the default folder by colab
!unzip mlops.zip
!rm -rf sample_data

✨ <font color="#f8c76b"> TASK </font> ✨

Let's check Artifactory to review the outcome.

On the Artifactory training instance, navigate to your newly created *remote HuggingfaceML repository*.

Or, Here is a direct link (replace x with your user number):  https://mlops17234464260.jfrog.io/artifactory/mlops-userx-hf-remote/

## Helper Functions

👀 The following cell defines some helper functions that will help us to test and develop the models.

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename



# Inference function

👀 The following cell defines the inference function. The "predict" function will get an image as input, and try to detect a human face in the image. 

In [ ]:
logging.getLogger("ultralytics").setLevel(logging.ERROR) 

model = YOLO(weights)

def infere(cheat=False, name=name):
    
    with open(config_file, 'r') as f:
        config = json.load(f)

    classes        = config['classes']
    target_classes = config['target_classes']
    conf           = config['conf']
    max_det        = config['max_det']

    filename = 'photo.jpg'  # Default filename
    if not cheat:
        filename = take_photo()
    else:
      data = "./model/main/img/tom.jpg"
      name = "Tom Hanks"
      shutil.copy(data, filename)
           

    frame = cv2.imread(filename) 

    frame_height, frame_width = frame.shape[:2]
    results = model.predict(source=frame, 
                            show=False, 
                            classes=classes, 
                            conf=conf,
                            max_det=max_det)

    # Extracting the names of detected classes
    boxes = results[0].boxes

     # Draw bounding boxes
    for box in boxes:
        label = model.names[int(box.cls)]
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convert to integer coordinates
  
        if int(box.cls) in target_classes:
          # Draw bounding box around detected object
          cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)  # Colored box 
          cv2.putText(frame, "Frog", (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
        else:
          cv2.rectangle(frame, (x1, y1), (x2, y2), (235, 222, 52), 3)  # Colored box 
          cv2.putText(frame, name, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,  (235, 222, 52), 2)

    cv2_imshow(frame)



Let's go ahead and test the model! 
Please approve using the camera for the model to work.
Once you have the video stream, click "Capture" (above the video block on the top left) to take a photo, and examine the model's output.
Then, run the cell again and take a photo of yourself and the Jfrog frog. Did the model identify the frog?

🚩**NOTE** 
The following cell may fail the first time because it runs in parallel to requesting access to the webcam. If it happens, just rerun the cell.

In [ ]:
try:
    infere()
except Exception as e:
    cv2.destroyAllWindows()
    raise e


In case you can't or don't want to use your own picture, you can use the cheat sheet cell below:

In [ ]:
try:
    infere(cheat=True)
except Exception as e:
    cv2.destroyAllWindows()
    raise e

---

# 🐸  <font color="#40be46">  Lab2: Securing models </font> 🐸 

## Block malicious models with Xray

Our Yolo model did quite well identifying human faces, but we wanted it to also detect frogs.
What can we do?
Searching HuggingFace, there is a model that seems just right! 

But is it safe? Let's configure Xray for our HuggingFace repository and find out. 

✨ <font color="#f8c76b"> TASK </font> ✨

Let's configure Xray to scan our HuggingFaceML remote repository.

#### Complete the following steps:

***Add the HuggingFaceML remote repository to the Xray index:***

1. Navigate to *Administration --> Xray Settings --> Indexed Resources --> Repositories*.
2. Search for your repository in the repositories list, and click on the ... menu. Click on `configure` then ensure that "Scan All Artifacts" is selected, then click on "Apply".
3. Select the ... menu again and click "Index Now". No need to change anything in the dialog box that opens. 
4. Lastly, again on the ... menu, select "Refresh Index Status" and make sure the status shows 1/1(100%) i.e. our initla model was scanned successfully.

For your convenience, we've already created a policy and a watch, so just adding your repository will be enough to kick off scanning. 
You can view the details here:

https://mlops17234464260.jfrog.io/ui/admin/xray/policiesGovernance/policies/edit/block-malicious-models?projectKey=mlops-user2

---

In [ ]:
try:
    snapshot_download(repo_id="MustEr/best_model_for_identifying_frogs")
except HfHubHTTPError as e:
    print("\n\n\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8 Xray blocked model download due to violation of the 'Malicious Package' policy.\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8\n\n")

### Check the scanning results 

✨ <font color="#f8c76b"> TASK </font> ✨

Navigate to your project's scans list (**remember to replace x with your student id**)

https://mlops17234464260.jfrog.io/ui/scans-list/repositories/mlops-userx-hf-remote-cache/scan-descendants?projectKey=mlops-userx

# 🐸  <font color="#40be46">  Lab3: Uploading an updated model to a local repository & deploying with Qwak </font> 🐸 

## Train the model to identify Frogs

Unfortunately, the "best_model_for_identifying_frogs" was not safe and we cannot use it.
But we still want to detect the frogs. Next, we will 'train' our original Yolo model to identify other objects, specifically frogs.

👀  Due to time constraints, our training function does not actually train on additional images. Instead, we'll just change the model configuration. Check the "config.json" file before and after the training to see the difference.


In [ ]:
def train(object_to_detect):

    if not object_to_detect in model.names.values():
        print(f"'{object_to_detect}' is not a valid YOLOv8 object. Hint: try Frog")
        return

    reverse_dict = {name: idx for idx, name in model.names.items()}
    class_id = reverse_dict.get(object_to_detect, None)

    with open(config_file, 'r') as file:
        config = json.load(file)

    target_classes = config['target_classes']

    # Add the new class number to the classes list if it's not already present
    if class_id not in config['classes']:
        config['classes'].append(class_id)
        config['classes'].extend([cls for cls in target_classes if cls not in config['classes']])

    config['max_det'] = 2

    # Save the updated config back to the file
    with open(config_file, 'w') as file:
        json.dump(config, file, indent=4)


In [ ]:
train("Frog")

## Run inference again

Let's check to see if the training did the trick!
Please take the JFrog frog and take a photo of the two of you together 😊 🐸 

In [ ]:
try:
    infere()
except Exception as e:
    cv2.destroyAllWindows()
    raise e


Again, a cheat sheet cell is available if you like in the next cell.

In [ ]:
try:
    infere(cheat=True)
except Exception as e:
    cv2.destroyAllWindows()
    raise e


## Upload to HF local

Now that we have a new, trained model, we need to upload it to the Artifactory HugginigFaceML local repository in order to share it with other teams and promote it towards Production.

✨ <font color="#f8c76b"> TASK </font>✨

Please perform the following steps:
1. Navigate to *Administration --> Repositories*.
2. From the projects dropdown list, select your project ("mlops-userx").  
3. Click 'Create a Repository' and select **Local**. Select HuggingFaceML.
4. On the next page, only provide the Repository Key (the repo name). The repository will be prefixed with your project name ("mlops-userx"). Please add the repository key **"hf-local"** then click Create Local Repository.
5. A dialog will open, suggesting to set up the HuggingFaceML client or Do It Later - click `Set Up HuggingFaceML client`.
6. Enter your Artifactory password then click `Generate Token & Create Instructions`.
7. Copy the *token* and paste it into the cell below, replacing the \<IDENTITY_TOKEN> placeholder.


In [ ]:
# Replace the <IDENTITY-TOKEN> placeholder with the token you generated in the JFrog Platform SetMeUp.
%env HF_TOKEN=<IDENTITY-TOKEN>

# Replace the <PATH> placeholder with the path to your ML Model Management repository in Artifactory, found in the JFrog Platform SetMeUp.
%env HF_ENDPOINT=https://mlops17234464260.jfrog.io/artifactory/api/huggingfaceml/mlops-$userid-hf-local

%env HF_HUB_DOWNLOAD_TIMEOUT=86400
%env HF_HUB_ETAG_TIMEOUT=86400

In [ ]:
from huggingface_hub import HfApi
import os

# Initialize API with the custom endpoint
api = HfApi(endpoint=os.getenv("HF_ENDPOINT"))

# Upload folder to the specified repository
api.upload_folder(
    folder_path=".",
    repo_id="frog-factor1",   # Replace with a name for your model
    repo_type="model"
)

### Check the results in Artifactory

✨ <font color="#f8c76b"> TASK </font> ✨

Let's check Artifactory to review the outcome.


1. On the Artifactory training instance, navigate to *Artifactory --> Artifacts* tab.
2. Find your newly created *local HuggingFaceML repository*.
3. Expand the repository and verify the YOLOV8 model is cached inside the repository, including the updated configuration file.

## Deploy with Qwak 

Now that we have a good model version, let's deploy it to a production endpoint with Qwak and monitor its performance.
Qwak is a fully managed end-to-end platform that contains the infrastructure AI practitioners need to build, deploy, manage and monitor GenAI, LLMs and classic ML in production.

We've already installed the Qwak SDK.

✨ <font color="#f8c76b"> TASK </font> ✨

1. Create a personal API key in the Qwak platform:
    - Go to [Qwak Platform](https://app.qwak.ai/), then login with your email and the password that was given to you for the training.
    - On the left hand side menu, Navigate to *Settings --> Personal API Keys*.
    - Click `Generate API Key`.
    - Copy the API key generated and replace the below <QWAK_PERSONAL_API_KEY> placeholder with it.

In [ ]:

# Replace <QWAK_PERSONAL_API_KEY> with your Qwak personal key from the qwak platform.
!qwak configure --api-key "<QWAK_PERSONAL_API_KEY>"

### Build the Qwak model

✨ <font color="#f8c76b"> TASK </font> ✨

In order to build and deploy the model through the Qwak platform, run the following commands.

In [ ]:
model_id = f"frog_factor_authenticator_{userid}"
!qwak models create "{model_id}" --project $userid       
!qwak models build --model-id $model_id ./model --base-image 'public.ecr.aws/w8k8y6b6/qwak-base:0.0.14-gpu-opencv' --gpu-compatible

### Check your model build status (can take up to 30 minutes)

✨ <font color="#f8c76b"> TASK </font> ✨

1. In the [Qwak Platform](https://app.qwak.ai/) Navigate to *Models*.
2. Select your project and click your model.
3. Under the *Builds* tab, identify your build and check the status.

## While we're waiting, let's do some detections with a model which is already on the Qwak platform.

We've already deployed the updated model into the Qwak platform in order to be able to test it without waiting for the build to complete. 
Let's check our deployment and see if we're able to authenticate with the Frog-Factor Authenticator.

## Some More Helper Functions

👀 The following cell defines some helper functions that adds some overlays based on the response we get from the Qwak model.

In [ ]:
from qwak_inference import RealTimeClient as QwakClient
from PIL import Image
import numpy as np 
import cv2

import numpy as np

def overlay_image_alpha(img, img_overlay, position="top"):
    """Overlay img_overlay on top of img at the specified position."""

    if position == "top":
        # Image ranges for top position
        y1, y2 = 0, min(img.shape[0], img_overlay.shape[0])
        x1, x2 = 0, min(img.shape[1], img_overlay.shape[1])
    elif position == "bottom":
        # Image ranges for middle-bottom position
        y1 = max(0, img.shape[0] - img_overlay.shape[0])
        y2 = img.shape[0]
        x1 = max(0, (img.shape[1] - img_overlay.shape[1]) // 2)
        x2 = min(img.shape[1], x1 + img_overlay.shape[1])
    else:
        raise ValueError("Position must be either 'top' or 'bottom'.")

    # Overlay ranges
    y1o, y2o = 0, min(img_overlay.shape[0], y2 - y1)
    x1o, x2o = 0, min(img_overlay.shape[1], x2 - x1)

    # Exit if nothing to do
    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return img

    # Blend overlay within the determined ranges
    img_crop = img[y1:y2, x1:x2]
    img_overlay_crop = img_overlay[y1o:y2o, x1o:x2o]

    # Split the alpha channel and the color channels
    if img_overlay_crop.shape[2] == 4:  # Ensure the overlay has an alpha channel
        img_overlay_color = img_overlay_crop[:, :, :3]
        alpha_mask = img_overlay_crop[:, :, 3] / 255.0

        # Reverse the color channels for BGR format
        img_overlay_color = img_overlay_color[:, :, ::-1]

        alpha_inv = 1.0 - alpha_mask

        for c in range(0, 3):
            img_crop[:, :, c] = (alpha_mask * img_overlay_color[:, :, c] +
                                 alpha_inv * img_crop[:, :, c])
    else:
        img_crop[:, :, :] = img_overlay_crop

    return img

def analze_results(results):

    with open(config_file, 'r') as f:
        config = json.load(f)

    target_classes = config['target_classes']

    #  Load the success image
    success_img = Image.open('model/main/img/success.png')
    success_rgba = np.array(success_img.convert('RGBA'))

    powered_by_img = Image.open('model/main/img/powered_by.png')
    powered_by_rgba_rgba = np.array(success_img.convert('RGBA')) 

    # Main processing loop
    target_detected = False

    frame = cv2.imread('photo.jpg')

    for result in results:
        box = result["box"]
        x1, y1, x2, y2 = map(int, box.values())  # Convert to integer coordinates

        if int(result["class"]) in target_classes:
            # Draw bounding box around detected object
            target_detected = True
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)  # Colored box 
            cv2.putText(frame, "Frog", (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        else:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (235, 222, 52), 3)  # Colored box 
            cv2.putText(frame, name, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (235, 222, 52), 2)

    if target_detected:
    # Overlay the success image at the top of the frame
        blended =overlay_image_alpha(frame, success_rgba)
        cv2_imshow(blended)
    else:
        # Display "Authentication Failed" text
        cv2.putText(frame, "Authentication Failed", (frame.shape[1] // 2 - 150, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
    cv2_imshow(frame)


def send_image_to_qwak(image):
    img = Image.open(image)
    image_rgb = img.convert('RGB')
    img_ndarray = np.array(image_rgb)
    img_list = img_ndarray.tolist()

    client = QwakClient(model_id="yolo_test_2")

    response = client.predict(img_list)

    results = response[0]["results"]

    return results



👀  We'll take another photo for the inference. Do not forget the frog! The detection boxes will not appear, we will perform the detection later on the Qwak platform.
If you wish to use Tom Hanks' photo instead, 

In [ ]:
imagefile = take_photo()
frame = cv2.imread('photo.jpg') 
cv2_imshow(frame)

In [ ]:
results = send_image_to_qwak('photo.jpg') # change to results = send_image_to_qwak('tom.jpg') if you wish to use Tom's picture
analze_results(results)

### Let's monitor the model performane

✨ <font color="#f8c76b"> TASK </font> ✨

1. In the [Qwak Platform](https://app.qwak.ai/) Navigate to *Models*.
2. Select "yolo" project and click "yolo-test-2"
3. Monitor the status under the "overview" tab.

# 🐸  <font color="#40be46">  Bonus Lab: Deploy your model to production endpoint in Qwak  </font> 🐸 

### Deploy your model

✨ <font color="#f8c76b"> TASK </font> ✨

1. In the [Qwak Platform](https://app.qwak.ai/) Navigate to *Models*.
2. Select your project and click your model.
3. Under the *Builds* tab, identify your build and ensure it finished building successfully, Then click `Deploy`. 
4. Select `Realtime`.
5. On the next screen, click `Deploy Model`. No need to change anything.
